# 多重ディスパッチ

このノートブックでは, Julia の鍵となる特徴の **多重ディスパッチ** を扱う.

多重ディスパッチは, ソフトウェアを *ジェネリック*, *高速* にする.

#### 使い慣れたものから始める

Julia における多重ディスパッチを理解するために, 今まで行ったことを振り返るところから始めよう.

Julia は, 関数が受け取る入力引数の型に関する情報を, Julia に与えずに、関数を宣言できる:

In [ ]:
f(x) = x^2

Julia は, どの入力引数の型が, 意味を成しているのかどうかを判断する.

In [ ]:
f(10)

In [ ]:
f([1, 2, 3])

#### 入力引数から, 型推定する

しかしながら, 入力引数にどのような型が許されているか, 明示的に, Julia へ伝えるために *option* も持つ

例として, string 型のみを入力引数に取る関数 `foo` を宣言してみよう. 

In [1]:
foo(x::String, y::String) = println("My inputs x and y are both strings!")

foo (generic function with 1 method)

`x` と `y` の型を, `String` 型に限定するために, 入力引数名に続けて, コロン二つ `::` と `String` を指定するだけである.

関数 `foo` が, `String` 型で動き, 他の入力引数の型では動かないことを示す.

In [2]:
foo("hello", "hi!")

My inputs x and y are both strings!


In [3]:
foo(3, 4)

LoadError: [91mMethodError: no method matching foo(::Int64, ::Int64)[39m

関数 `foo` が integer (`Int`) 型の入力で作動する様にするために, 関数 `foo` を宣言するときに入力引数へ, `::Int` を指定しよう.

In [4]:
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

foo (generic function with 2 methods)

In [5]:
foo(3, 4)

My inputs x and y are both integers!


関数 `foo` は, integer 型において作動した. しかし, 関数 `foo` は, `x` と `y` が string 型の時も依然として作動する.

In [6]:
foo("hello", "hi!")

My inputs x and y are both strings!


これが, 多重ディスパッチの核心に至る始まりである. 以下のように宣言した際,

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```

上書きや置き換えを行っていない.

```julia
foo(y::String, y::String)
```

代わりに, 関数 `foo` によってよばれる***generic 関数***へ, 追加の ***メソッド*** を加えたのみである. 

***generic 関数*** は, 特定の操作に関連付けられた抽象概念である.

例えば, generic 関数 `+` は, 加算のコンセプトを象徴する.

***メソッド*** は, *特定の引数型* のための generic 関数の明確な実装である.

例えば, `+` は, 浮動小数点, integer, 行列などを受け入れるメソッドを持つ.

関数 `methods()` を用いることで, 例えば, 関数 `foo` がどれほどのメソッドを持っているか確認することができる.

In [ ]:
methods(foo)

そのため, integer や string に対して関数 `foo` をよぶことができた. 引数の特定の集合において関数 `foo` をよぶとき, Julia は, 入力の型を推定し, 適切なメソッドを, ディスパッチする. *これ*が, 多重ディスパッチ.

多重ディスパッチは, コードを generic にそして速くする. 多重ディスパッチを活用した, コードは generic かつ 柔軟である. なぜなら, 具体的な実装ではなく, 加算や乗算などの抽象的な操作の観点からコードを記述することができるためである.

同時に, 素早くコードが実行される, Julia は, 関連する型に対して効率的なメソッドを呼び出すことができるから.

generic 関数を呼び出したときに, どのメソッドがディスパッチされたのか知るために, `@which` マクロを利用できる:

In [ ]:
@which foo(3, 4)

そして, generic 関数 `foo` へさらに他のメソッドを追加することができる.
サブタイプとして, `Int`, `Float64`, そして, 数に関わると想定されるその他のオブジェクトすべてを, ***抽象型*** `Number` へと, とるメソッドを追加する.

In [ ]:
foo(x::Number, y::Number) = println("My inputs x and y are both numbers!")

関数 `foo` へのこのメソッドは, 機能する. 例えば, 浮動小数点数に対して:

In [ ]:
foo(3.0, 4.0)

さらに, 関数 `foo` への全ての型を入力として受け取れる, ダックタイプドメソッドにたいして, フォールバックを追加することもできる.

In [2]:
foo(x, y) = println("I accept inputs of any type!")

foo (generic function with 1 method)

これまで, `foo` のために書いてきたメソッドを考慮すると, このメソッドは、非数を `foo` に渡すたびに呼び出される:

In [3]:
foo(true, true)

I accept inputs of any type!


### 演習

#### 9.1

関数 `foo` を拡張します, ひとつのみ, `Bool` 型を入力引数として受け取り, "foo with one boolean!" と出力するメソッドを追加してください.

#### 9.2

上記で書いたメソッドが, 実行時にディスパッチされていることを確認してください

```julia
foo(true)
```
